In [13]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split

In [3]:
# Leer CSV
df = pd.read_csv('./data/country_CR_train.csv')
df.head()

,content,sentiment
0,@NoilyMV yo soy totalmente puntual,NONE
1,@SandraCauffman Hola Sandrita. No le habia des...,P
2,Si andan haciendo eso mejor se quedaran callad...,N
3,Que pereza quiero choco banano,N
4,"@robertobrenes Bueno, no es tanto lo mayor com...",N


In [4]:
# Sentimientos únicos
pd.DataFrame(df['sentiment'].unique(), columns=['Sentiments Unique'])

,Sentiments Unique
0,NONE
1,P
2,N
3,NEU


In [5]:
# Conversión de sentiments a variables numéricas
df['sentiment'] = df.sentiment.map({'NONE':0, 'N':1, 'NEU':2, 'P':3})

# Vizualización de los sentimientos modificados
df

,content,sentiment
0,@NoilyMV yo soy totalmente puntual,0
1,@SandraCauffman Hola Sandrita. No le habia des...,3
2,Si andan haciendo eso mejor se quedaran callad...,1
3,Que pereza quiero choco banano,1
4,"@robertobrenes Bueno, no es tanto lo mayor com...",1
...,...,...
772,Quiero como tener carro para hacer todo rápido,2
773,Tristeza más hp me da siempre en año nuevo,1
774,no encuentro ninguna buena serie en NF.,1
775,@Alphasniper97 Alpha suerte y ten un feliz año...,3


In [6]:
wordCount = []
# Conteo de palabras
for i in df.content.tolist():
    wordCount.append(len(i.split()))
df = df.assign(wordcount = wordCount)
df

,content,sentiment,wordcount
0,@NoilyMV yo soy totalmente puntual,0,5
1,@SandraCauffman Hola Sandrita. No le habia des...,3,25
2,Si andan haciendo eso mejor se quedaran callad...,1,22
3,Que pereza quiero choco banano,1,5
4,"@robertobrenes Bueno, no es tanto lo mayor com...",1,24
...,...,...,...
772,Quiero como tener carro para hacer todo rápido,2,8
773,Tristeza más hp me da siempre en año nuevo,1,9
774,no encuentro ninguna buena serie en NF.,1,7
775,@Alphasniper97 Alpha suerte y ten un feliz año...,3,9


In [32]:
# Preparación de los datos
X = df[['wordcount']].values
y = df['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [33]:
# Predicción del modelo
print(clf.predict(y_test))

ValueError: Expected 2D array, got 1D array instead:
array=[0. 3. 0. 2. 3. 2. 1. 3. 1. 1. 1. 0. 1. 2. 0. 1. 0. 1. 1. 3. 1. 1. 3. 0.
 1. 1. 0. 2. 0. 0. 1. 2. 0. 0. 1. 3. 3. 2. 3. 0. 0. 3. 3. 3. 1. 3. 3. 3.
 1. 1. 0. 3. 0. 3. 3. 1. 1. 3. 0. 1. 0. 3. 1. 1. 3. 0. 0. 2. 1. 1. 1. 3.
 0. 3. 3. 0. 3. 0. 2. 2. 1. 3. 1. 1. 0. 3. 1. 2. 3. 1. 1. 3. 3. 1. 1. 3.
 1. 1. 0. 1. 1. 1. 3. 1. 0. 1. 3. 1. 3. 3. 3. 1. 1. 3. 0. 1. 0. 0. 1. 2.
 0. 0. 0. 1. 3. 2. 3. 0. 1. 1. 3. 1. 1. 3. 3. 0. 2. 2. 1. 3. 1. 1. 3. 1.
 3. 0. 2. 1. 1. 0. 3. 1. 1. 1. 2. 1. 0. 3. 3. 1. 3. 0. 0. 1. 1. 2. 1. 2.
 1. 1. 0. 3. 3. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 3. 1. 1. 3. 1.
 3. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.